In [45]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
import time
import importlib
import pandas as pd
import numpy as np
import pickle
import p2 as nn  
importlib.reload(nn)

<module 'p2' from 'C:\\Users\\vipul\\OneDrive\\Documents\\MPCS 53111 Machine Learning\\HW4\\hw4\\p2.py'>

### Brief overview of your tasks in the accompanying Python script.  Please also see the outline in the homework instructions.
- You need to complete the neural network by fill in codes where there is a #todo prompt.
- **The architecture of the neural network you are building is a vanilla version of that of Pytorch/Tensorflow consisting of nodes of *Operation Classes*. Each *Operation Class* has a forward method (to calculate the operation result stored in self.value) and a backward method (to calculate the gradient of the operation w.r.t. the final loss function).**
- You need to implement forward and backward methods for *Operation Classes* Mul, VDot, Sigmoid, Relu (not required), Softmax, Log. Add, Aref, and Accuracy has been implemented for you for reference
- self.params is the list to store the trainable parameters (objects of Class Param). set_weights(weights) has been implemented for you where the provided weights and biases are converted to Param objects and stored in self.params. You need to study the code and implement <code>get_weights()</code> and <code>init_weights_with_xavier()</code>
- self.components is the list to natively mimic the function of the computational graph. Helper functions nn_unary_op(op, x) and nn_binary_op(op, x, y) are provided to facilitate creating an operation and adding it to the computational graph. For example, instead of $a=b+c$, you should use <code>a = self.nn_binary_op(Add, Value(b), Value(c))</code>. Only in this way you can create an operation object <code>a</code> with <code>a.value</code> and <code>a.grad</code> to support the forward and backward method of the neural network.
- Placeholder including <code>self.sample_placeholder</code>, <code>self.label_placeholder</code>, <code>self.pred_placeholder</code>, <code>self.loss_placeholder</code>, <code>self.accy_placeholder</code> are all empty vectors that will be assigned values when executing <code>forward</code> or <code>backward</code>. They facilitate the construcation of the computational graph. <code>self.sample_placeholder</code> is the input to the NN. We feed different examples by calling <code>self.sample_placeholder.set(X[i])</code> and <code>self.label_placeholder.set(y[i])</code>. <code>self.pred_placeholder</code>, <code>self.loss_placeholder</code>, <code>self.accy_placeholder</code> changes values in each iteration in <code>fit</code>.
- <code>self.forward()</code> is provided for you where each operation object in self.components are evalued from the begining to the end.
- <code>self.backward()</code> is provided for you where derivative of each operation object in self.components are calculated from the end (loss function) to the beginning.
- You need to implement sgd_update_parameter.
- You could implement gradient_estimate to debug.
- A test function test_set_and_get_weights() has been provided for you to test <code>self.get_weights</code>. Feel free to create more test functions.

### Below is the test code for get_weights

In [46]:
# You should expect the following message
nn.test_set_and_get_weights()

Passed the test for set_weights and get_weights.


### Below are test cases to help you debug the different operation Classes

In [47]:

m1 = nn.InputValue(np.arange(12).reshape((3,4)))
m2 = nn.InputValue(np.arange(12,24).reshape((3,4)))
m3 = nn.InputValue(np.arange(24,36).reshape((3,4)))
m4 = nn.InputValue(np.arange(36,48).reshape((3,4)))

v1 = nn.InputValue(np.arange(3).reshape((3,)))
v2 = nn.InputValue(np.arange(3,6).reshape((3,)))
v3 = nn.InputValue(np.arange(6,9).reshape((3,)))
v4 = nn.InputValue(np.arange(4).reshape((4,)))

In [48]:
#Test VDot
x = nn.Mul(m1, m2)
y = nn.VDot(v1, x)
z = nn.Mul(y, v4)
x.forward()
y.forward()
z.forward()
z.grad = np.ones_like(z.value)
y.grad = 0
x.grad = 0
v1.grad = 0
z.backward()
y.backward()
x.backward()

yvalue = np.array([384., 463., 548., 639.])
ygrad = np.array([0., 1., 2., 3.])
yagrad = np.array([ 204.,  700., 1388.])
ybgrad = np.array([[0., 0., 0., 0.],
                   [0., 1., 2., 3.],
                   [0., 2., 4., 6.]])

if not np.array_equal(y.value, yvalue):
    raise FailTestError("y.value not equal to matrix product of x.value and v1.value")
if not np.array_equal(y.grad, ygrad):
    raise FailTestError("gradient of y is incorrect")
if not np.array_equal(y.a.grad, yagrad):
    raise FailTestError("gradient of a in y is incorrect")
if not np.array_equal(y.b.grad, ybgrad):
    raise FailTestError("gradient of b in y is incorrect")
print("Passed Test on VDot")

Passed Test on VDot


In [49]:
# Test Sigmoid
x = nn.Add(v1, v2)
y = nn.Sigmoid(x)
z = nn.Mul(y, v3)
x.forward()
y.forward()
z.forward()
z.grad = np.ones_like(z.value)
y.grad = 0
x.grad = 0
z.backward()
y.backward()
x.backward()

yvalue = np.array([0.95257413, 0.9933072 , 0.999089], dtype=np.float32)
yagrad = np.array([0.2710599 , 0.04653623, 0.00728134], dtype=np.float32)

if not np.array_equal(np.round(y.value, 5), np.round(yvalue, 5)):
    raise FailTestError("y.value not equal to sigmoid of x")
if not np.array_equal(np.round(y.a.grad, 5), np.round(yagrad, 5)):
    raise FailTestError("gradient of a in y is incorrect")
print("Passed Test on Sigmoid")

Passed Test on Sigmoid


In [50]:
# Test RELU
# THIS IS OPTIONAL.
x = nn.Add(v1, v2)
y = nn.RELU(x)
z = nn.Mul(y, v3)
x.forward()
y.forward()
z.forward()
z.grad = np.ones_like(z.value)
y.grad = 0
x.grad = 0
z.backward()
y.backward()
x.backward()

yvalue = np.array([3., 5., 7.], dtype=np.float32)
yagrad = np.array([6., 7., 8.], dtype=np.float32)

print("Passing this teset is optional.")

if not np.array_equal(np.round(y.value, 5), np.round(yvalue, 5)):
    raise FailTestError("y.value not equal to Relu of x")
if not np.array_equal(np.round(y.a.grad, 5), np.round(yagrad, 5)):
    raise FailTestError("gradient of a in y is incorrect")
print("Passed Test on RELU")

Passing this teset is optional.
Passed Test on RELU


In [51]:
# Test SoftMax
x = nn.Add(v1, v2)
y = nn.SoftMax(x)
z = nn.Mul(y, v3)
x.forward()
y.forward()
z.forward()
z.grad = np.ones_like(z.value)
y.grad = 0
x.grad = 0
z.backward()
y.backward()
x.backward()

yvalue = np.array([0.01587624, 0.11731043, 0.86681336], dtype=np.float32)
yagrad = np.array([-0.02938593, -0.09982383,  0.12920949], dtype=np.float32)

if not np.array_equal(np.round(y.value, 5), np.round(yvalue, 5)):
    raise FailTestError("y.value not equal to SoftMax of x")
if not np.array_equal(np.round(y.a.grad, 5), np.round(yagrad, 5)):
    raise FailTestError("gradient of a in y is incorrect")
print("Passed Test on SoftMax")

Passed Test on SoftMax


In [52]:
# Test Log
x = nn.Add(v1, v2)
y = nn.Log(x)
z = nn.Mul(y, v3)
x.forward()
y.forward()
z.forward()
z.grad = np.ones_like(z.value)
y.grad = 0
x.grad = 0
z.backward()
y.backward()
x.backward()

yvalue = np.array([1.0986123, 1.609438 , 1.9459102], dtype=np.float32)
yagrad = np.array([2.       , 1.4      , 1.1428572], dtype=np.float32)

if not np.array_equal(np.round(y.value, 5), np.round(yvalue, 5)):
    raise FailTestError("y.value not equal to log of x")
if not np.array_equal(np.round(y.a.grad, 5), np.round(yagrad, 5)):
    raise FailTestError("gradient of a in y is incorrect")
print("Passed Test on Log")

Passed Test on Log


## Applying to the MNIST dataset
- You should use the MNIST dataset to test the neural network you build above.

In [53]:
# we will use train.csv for training and testing as it test.csv doesn't contain label
data = pd.read_csv("./data/train.csv")
train_data = data.iloc[:30000]  # 30000
test_data = data.iloc[30000:]  # 12000

pixel_columns = [f"pixel{i}" for i in range(784)]

# normalize by dividing by 255 as the pixel ranges from 0 to 255
train_x = train_data[pixel_columns].values.astype(nn.DATA_TYPE)/255
train_y = train_data["label"].values.astype(nn.DATA_TYPE)

test_x = test_data[pixel_columns].values.astype(nn.DATA_TYPE)/255
test_y = test_data["label"].values.astype(nn.DATA_TYPE)

### Debugging the fit function

In [54]:
importlib.reload(nn)  # important line so that the changes you made on p2.py will be reflected without restarting the kernel
# Here we use the 2-layer NN with 1 hidden layer, feel free to experiment on your own
nodes_array = [784, 128, 10]
model = nn.NN(nodes_array, "sigmoid")

class FailTestError(Exception):
    'Raised when a test fails'
    pass

# You can use the provided sample weights for initialization to help debug
with open("./data/sample_weights.pkl", 'rb') as f:
    weights = pickle.load(f)
model.set_weights(weights)

# You can use the first 2 samples to test if the gradients are correct
X = train_x[:2]
y = train_y[:2]

# when calling fit, a computational graph will be built first, you should expect the exact lines printed
model.fit(X, y, alpha=0.01, t=1)

# Load the sample gradient for debugging
with open("./data/sample_gradient.pkl", 'rb') as f:
    sample_grad = pickle.load(f)

# first layer's weight of shape (784, 128)
if not np.allclose(model.params[0].grad, sample_grad["w1"]):
    raise FailTestError("gradient of the first layer's weight is incorrect")
# first layer's bias of shape (128, )
if not np.allclose(model.params[1].grad, sample_grad["b1"]):
    raise FailTestError("gradient of the first layer's bias is incorrect")
# second layer's weight of shape (128, 10)
if not np.allclose(model.params[2].grad, sample_grad["w2"]):
    raise FailTestError("gradient of the second layer's weight is incorrect")
# second layer's bias of shape (10, )
if not np.allclose(model.params[3].grad, sample_grad["b2"]):
    raise FailTestError("gradient of the second layer's bias is incorrect")
print("Congrats! You have passed the test of your fit function, your NN model should be good to go!")

<class 'p2.Param'> <class 'p2.Param'>
Append <VDot> to the computational graph
Append <Add> to the computational graph
Append <Sigmoid> to the computational graph
<class 'p2.Param'> <class 'p2.Param'>
Append <VDot> to the computational graph
Append <Add> to the computational graph
Append <SoftMax> to the computational graph
Append <Aref> to the computational graph
Append <Log> to the computational graph
Append <Mul> to the computational graph
Append <Accuracy> to the computational graph


C:\Users\vipul\OneDrive\Documents\MPCS 53111 Machine Learning\HW4\hw4\p2.py:351: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  weights = np.array(weights)


  0%|          | 0/2 [00:00<?, ?it/s]

Epoch 0: train loss = 2.4059, accy = 0.5000, [0.024 secs]
Congrats! You have passed the test of your fit function, your NN model should be good to go!


### Now train your NN on the whole training dataset

In [55]:
importlib.reload(nn)  # important line so that the changes you made on p2.py will be reflected without restarting the kernel
# Here we use the 2-layer NN with 1 hidden layer, feel free to experiment on your own
nodes_array = [784, 128, 10]
model = nn.NN(nodes_array, "sigmoid")
model.init_weights_with_xavier()
model.fit(train_x, train_y, 0.01, 10)

<class 'p2.Param'> <class 'p2.Param'>
Append <VDot> to the computational graph
Append <Add> to the computational graph
Append <Sigmoid> to the computational graph
<class 'p2.Param'> <class 'p2.Param'>
Append <VDot> to the computational graph
Append <Add> to the computational graph
Append <SoftMax> to the computational graph
Append <Aref> to the computational graph
Append <Log> to the computational graph
Append <Mul> to the computational graph
Append <Accuracy> to the computational graph


  0%|          | 0/30000 [00:00<?, ?it/s]

Epoch 0: train loss = 0.3100, accy = 0.9109, [72.511 secs]


  0%|          | 0/30000 [00:00<?, ?it/s]

Epoch 1: train loss = 0.2371, accy = 0.9319, [72.628 secs]


...

Epoch 7: train loss = 0.0919, accy = 0.9740, [72.033 secs]


  0%|          | 0/30000 [00:00<?, ?it/s]

Epoch 8: train loss = 0.0818, accy = 0.9776, [71.493 secs]


  0%|          | 0/30000 [00:00<?, ?it/s]

Epoch 9: train loss = 0.0733, accy = 0.9800, [71.549 secs]


In [56]:
# After 10 epochs of training, you should expect an accuracy over 95% and loss around 0.1
accy, loss = model.eval(test_x, test_y)
print("Test accuracy = %.4f, accy = %.4f" % (accy, loss))

Test accuracy = 0.9637, accy = 0.1166


## Relu

In [57]:
importlib.reload(nn)  # important line so that the changes you made on p2.py will be reflected without restarting the kernel
# Here we use the 2-layer NN with 1 hidden layer, feel free to experiment on your own
nodes_array = [784, 128, 10]
model = nn.NN(nodes_array, "relu")
model.init_weights_with_xavier()
model.fit(train_x, train_y, 0.01, 10)

<class 'p2.Param'> <class 'p2.Param'>
Append <VDot> to the computational graph
Append <Add> to the computational graph
Append <RELU> to the computational graph
<class 'p2.Param'> <class 'p2.Param'>
Append <VDot> to the computational graph
Append <Add> to the computational graph
Append <SoftMax> to the computational graph
Append <Aref> to the computational graph
Append <Log> to the computational graph
Append <Mul> to the computational graph
Append <Accuracy> to the computational graph


  0%|          | 0/30000 [00:00<?, ?it/s]

Epoch 0: train loss = 0.1574, accy = 0.9525, [87.602 secs]


  0%|          | 0/30000 [00:00<?, ?it/s]

Epoch 1: train loss = 0.1008, accy = 0.9687, [86.883 secs]


...

Epoch 7: train loss = 0.0167, accy = 0.9955, [87.063 secs]


  0%|          | 0/30000 [00:00<?, ?it/s]

Epoch 8: train loss = 0.0142, accy = 0.9963, [87.171 secs]


  0%|          | 0/30000 [00:00<?, ?it/s]

Epoch 9: train loss = 0.0116, accy = 0.9971, [87.381 secs]


In [58]:
# After 10 epochs of training, you should expect an accuracy over 95% and loss around 0.1
accy, loss = model.eval(test_x, test_y)
print("Test accuracy = %.4f, accy = %.4f" % (accy, loss))

Test accuracy = 0.9735, accy = 0.1052
